# <span style="color:deepskyblue">1.추천 시스템의 개요와 배경 </span>    
---

## <span style="color:lightpink">추천 시스템의 유형</span> 

콘테츠 기반 필터링  
협업 필터링- 최근접 이웃 협업 필터링, 잠재 요인 협업 필터링

# <span style="color:deepskyblue">2.콘테츠 기반 필터링 추천 시스템 </span>    
---

사용자가 특정한 아이템을 매우 선호하는 경우 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천

# <span style="color:deepskyblue">3.최근접 이웃 협업 필터링 </span>    
---

사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행

축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가하는 것이 목표

- 행렬데이터 사용  
행: 개별 사용자  
열: 개별 아이템